This is a stock price prediction project that basically copies from this website:https://medium.com/@Currie32/predicting-the-stock-market-with-the-news-and-deep-learning-7fc8f5f639bc
full project github: https://github.com/Currie32/Predicting-the-Dow-Jones-with-Headlines
model description: https://www.aclweb.org/anthology/C16-1229.pdf
Primarily uses Keras
This project is mainly for me to familiarize myself in using Keras libraries and packages. It is also a way for me to learning about word embedding and some basic ideas about NLP. 

In [72]:
import pandas as pd
import numpy as np
import tensorflow as tf 
import re 
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import accuracy_score as acc


from keras.models import Sequential
from keras import initializers
from keras.layers import Dropout, Activation, Embedding, Convolution1D
from keras.layers import MaxPooling1D, Input, Dense, BatchNormalization, Flatten, Reshape, Concatenate

from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
from keras import regularizers 

In [8]:
dj = pd.read_csv("dataset/DowJones.csv")
news = pd.read_csv("dataset/RedditNews.csv")

In [9]:
dj.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234


In [10]:
dj.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Volume       0
Adj Close    0
dtype: int64

In [11]:
news.isnull().sum()

Date    0
News    0
dtype: int64

In [12]:
news.head()

,Date,News
0,2016-07-01,A 117-year-old woman in Mexico City finally re...
1,2016-07-01,IMF chief backs Athens as permanent Olympic host
2,2016-07-01,"The president of France says if Brexit won, so..."
3,2016-07-01,British Man Who Must Give Police 24 Hours' Not...
4,2016-07-01,100+ Nobel laureates urge Greenpeace to stop o...


In [13]:
print(dj.shape)
print(news.shape)

(1989, 7)
(73608, 2)


In [14]:
news = news[news.Date.isin(dj.Date)]

In [15]:
print(len(set(dj.Date)))
print(len(set(news.Date)))

1989
1989


In [16]:
#here the dj becomes the difference between the opening price of the day and the next date
# set_index first so that when .diff calculate the difference in price it won't effect the date
dj = dj.set_index('Date').diff(periods=1)
dj['Date'] = dj.index
dj = dj.reset_index(drop=True)

#drop unnecessary columns
dj = dj.drop(['High','Low','Close','Volume','Adj Close'],1)


In [17]:
dj

,Open,Date
0,NaN,2016-07-01
1,-211.480468,2016-06-30
2,-256.740235,2016-06-29
3,-265.509765,2016-06-28
4,164.701172,2016-06-27
...,...,...
1984,-79.139649,2008-08-14
1985,100.739258,2008-08-13
1986,148.890625,2008-08-12
1987,-52.030273,2008-08-11


In [18]:
dj = dj[dj.Open.notnull()]

In [19]:
price = []
headlines = []

#row now is an index series, where the series is of the form [Open, Date] pair, therefore call row[1] to only get the series, which is the value of each row
for row in dj.iterrows():
    daily_headlines = []
    date = row[1]["Date"]
    price.append(row[1]['Open'])
    # collect news from the same date and put them into daily_headlines
    for row_ in news[news.Date==date].iterrows():
        daily_headlines.append(row_[1]['News'])
    headlines.append(daily_headlines)

In [20]:
print(len(price))
print(len(headlines))

1988
1988


In [21]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [22]:
def clean_text(text, remove_stopwords = True):
    
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'0,0', '00', text) 
    text = re.sub(r'[_"\-;%()|.,+&=*%.,!?:#@\[\]]', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'\$', ' $ ', text)
    text = re.sub(r'u s ', ' united states ', text)
    text = re.sub(r'u n ', ' united nations ', text)
    text = re.sub(r'u k ', ' united kingdom ', text)
    text = re.sub(r'j k ', ' jk ', text)
    text = re.sub(r' s ', ' ', text)
    text = re.sub(r' yr ', ' year ', text)
    text = re.sub(r' l g b t ', ' lgbt ', text)
    text = re.sub(r'0km ', '0 km ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)
    return text


In [23]:
# Clean the headlines
clean_headlines = []

for daily_headlines in headlines:
    clean_daily_headlines = []
    for headline in daily_headlines:
        clean_daily_headlines.append(clean_text(headline))
    clean_headlines.append(clean_daily_headlines)


In [24]:
clean_headlines[0]

['jamaica proposes marijuana dispensers tourists airports following legalisation kiosks desks would give people license purchase 2 ounces drug use stay',
 'stephen hawking says pollution stupidity still biggest threats mankind certainly become less greedy less stupid treatment environment past decade',
 'boris johnson says run tory party leadership',
 'six gay men ivory coast abused forced flee homes pictured signing condolence book victims recent attack gay nightclub florida',
 'switzerland denies citizenship muslim immigrant girls refused swim boys report',
 'palestinian terrorist stabs israeli teen girl death bedroom',
 'puerto rico default $ 1 billion debt friday',
 'republic ireland fans awarded medal sportsmanship paris mayor',
 'afghan suicide bomber kills 40 bbc news',
 'us airstrikes kill least 250 isis fighters convoy outside fallujah official says',
 'turkish cop took istanbul gunman hailed hero',
 'cannabis compounds could treat alzheimer removing plaque forming proteins br

In [25]:
# Find the number of times each word was used and the size of the vocabulary
word_counts = {}

for date in clean_headlines:
    for headline in date:
        for word in headline.split():
            if word not in word_counts:
                word_counts[word] = 1
            else:
                word_counts[word] += 1
            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 35190


In [26]:
# load GLoVe's embeddings
embeddings_index = {}
with open('C:/Users/edwar/Projects/Stock-Prediction/dataset/glove.840B.300d.txt', encoding ='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding
    f.close()        
print("word embeddings:", len(embeddings_index))

word embeddings: 2196016


In [27]:
# Find the number of words that are missing from GloVe, and are used more than our threshold.
missing_words = 0
threshold = 10

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from GloVe:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from GloVe: 47
Percent of words that are missing from vocabulary: 0.13%


In [28]:
# Use only the words that appear more than a set threashold or that appears in GLoVe.
# creates two dictionary that gives indexing value to each word that is used
 
value = 0
vocab_to_int = {}

for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value 
        value += 1
        
# Special tokens that will be added to our vocab <UNK> = unknown, <PAD> = 
codes = ["<UNK>","<PAD>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total Number of Unique Words:", len(word_counts))
print("Number of Words we will use:", len(vocab_to_int))
print("Percent of Words we will use: {}%".format(usage_ratio))    


Total Number of Unique Words: 35190
Number of Words we will use: 31265
Percent of Words we will use: 88.85%


In [29]:
for code in codes:
    print(vocab_to_int[code])

31263
31264


In [30]:
#Create New embeddings for words in word_counts that are not in embedding_index 

embedding_dim = 300

#total number of words to use in training

nb_words = len(vocab_to_int)

#create a embedding matrix, each row is for a word, with total of 300 columns for the dimension of the GloVe vectors

word_embedding_matrix = np.zeros((nb_words,embedding_dim))

for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        #create a new embedding vectors with random values for words not in GLoVe embedding
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        # we also add this newly generated random embedding to the embedding_index along with the rest of the GloVe embedding
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding
        
print(len(word_embedding_matrix))

31265


All the words in the embedding "including the pretrained ones" will be updated as the model train. 

In [31]:
# now to create a replicated headlines where all the words are transformed into their regarding index in vocab_to_int 
# int_headlines: all the headlines
# int_daily_headlines: all the headlines in a day
# int_headline: single headline in a day 
# int_headlines[int_daily_headlines[int_headline]]

# for words in the headlines that are not in vocab_to_int 
# (meaning the word didn't hit the threshold and isn't in the pretrained embedding index)
# we replace it with the <UNK> token for "unknown"
word_count = 0
unk_count = 0 

int_headlines = []

for date in clean_headlines:
    int_daily_headlines = []
    for headline in date:
        int_headline = []
        for word in headline.split():
            word_count += 1
            if word in vocab_to_int:
                int_headline.append(vocab_to_int[word])
            else:
                int_headline.append(vocab_to_int["<UNK>"])
                unk_count += 1
        int_daily_headlines.append(int_headline)
    int_headlines.append(int_daily_headlines)

unk_percent = round((unk_count/word_count)*100,4)

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))





Total number of words in headlines: 615989
Total number of UNKs in headlines: 5262
Percent of words that are UNK: 0.8542%


In [32]:
#length of individual headline
lengths = []
for date in int_headlines:
    for headline in date:
        lengths.append(len(headline))

lengths = pd.DataFrame(lengths, columns=['count'])

In [33]:
lengths.describe()

,count
count,49693.000000
mean,12.395891
std,6.790246
min,1.000000
25%,7.000000
50%,10.000000
75%,16.000000
max,41.000000


In [34]:
# limiting the length of total words per day to 200 words
# and headlines to 16 words to reduce training time

max_headline_length = 16
max_daily_length = 200
# pad_headlines only divides up the news up to date, not individual news (2D)
# for news length that does not reach the max_daily_length, we pad it with the <PAD> token 

pad_headlines = []

for date in int_headlines:
    pad_daily_headlines = []
    for headline in date:
        #add the headline to pad daily headline if headline is less than max length
        if len(headline) <= max_headline_length:
            for word in headline:
                #needs to append word by word so pad_daily_headline is 1D
                pad_daily_headlines.append(word)
        else:
            headline = headline[:max_headline_length]
            for word in headline:
                pad_daily_headlines.append(word)
    if len(pad_daily_headlines) < max_daily_length:
        for i in range(max_daily_length - len(pad_daily_headlines)):
            pad = vocab_to_int["<PAD>"]
            pad_daily_headlines.append(pad)
    else:
        pad_daily_headlines = pad_daily_headlines[:max_daily_length]
    pad_headlines.append(pad_daily_headlines)
        

In [35]:
# normalize (between [0,1]) the opening prices(target values, response variable)
max_price = max(price)
min_price = min(price)
mean_price = np.mean(price)

def normalize(price):
    return ((price-min_price)/(max_price-min_price))

In [36]:
norm_price = []
for p in price:
    norm_price.append(normalize(p))

In [37]:
# checking normalization
print(min(norm_price))
print(max(norm_price))
print(np.mean(norm_price))

0.0
1.0
0.5448422454901358


In [38]:
# create train test split


x_train, x_test, y_train, y_test = train_test_split(pad_headlines, norm_price, test_size=0.15, random_state=2)

x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [34]:
# Check the lengths of train and test set
print(len(x_train))
print(len(x_test))

1689
299


In [66]:
filter_length1 = 3
filter_length2 = 5
dropout = 0.5
learning_rate= 0.001
weights = initializers.TruncatedNormal(mean=0.0, stddev=0.1, seed=2)
nb_filter = 16
rnn_output_size = 128
hidden_dims = 128
wider = True
deeper = True

if wider == True:
    nb_filter *= 2
    rnn_output_size *= 2
    hidden_dims *= 2

def build_model():
    model1 = Sequential()
    
    model1.add(Embedding(nb_words,
                        embedding_dim,
                        weights=[word_embedding_matrix],
                        input_length=max_daily_length))
    model1.add(Dropout(dropout))
    model1.add(Convolution1D(filters = nb_filter,
                            kernel_size=filter_length1,
                            padding = 'same',
                            activation = 'relu'))
    model1.add(Dropout(dropout))
    if deeper == True:
        model1.add(Convolution1D(filters = nb_filter,
                                kernel_size = filter_length1,
                                padding = 'same',
                                activation = 'relu'))
        model1.add(Dropout(dropout))
    
    model1.add(LSTM(rnn_output_size,
                   activation = None,
                   kernel_initializer = weights,
                   dropout = dropout))
    
    ######
    
#     model2 = Sequential()
    
#     model2.add(Embedding(nb_words,
#                         embedding_dim,
#                         weights = [word_embedding_matrix],
#                         input_length = max_daily_length))
#     model2.add(Dropout(dropout))
    
#     model2.add(Convolution1D(filters = nb_filter,
#                             kernel_size = filter_length2,
#                             padding = 'same',
#                             activation = 'relu'))
    
#     model2.add(Dropout(dropout))
    
#     if deeper == True:
#         model2.add(Convolution1D(filters = nb_filter,
#                                  kernel_size = filter_length2,
#                                  padding = 'same',
#                                  activation = 'relu'))
#         model2.add(Dropout(dropout))
        
#     model2.add(LSTM(rnn_output_size,
#                     activation=None,
#                     kernel_initializer=weights,
#                     dropout = dropout))
    
    #####
    
    
    model = Sequential()
    model.add(model1)
    
    model.add(Dense(hidden_dims, kernel_initializer=weights))
    model.add(Dropout(dropout))
    
    if deeper == True:
        model.add(Dense(hidden_dims//2, kernel_initializer=weights))
        model.add(Dropout(dropout))
        
    model.add(Dense(1,
                   kernel_initializer=weights,
                    name='output'))
    
    model.compile(loss='mean_squared_error',
                 optimizer=Adam(lr=learning_rate,clipvalue=1.0))
    return model


In [67]:
for deeper in [False]:
    for wider in [True,False]:
        for learning_rate in [0.001]:
            for dropout in [0.3, 0.5]:
                model = build_model()
                print()
                print("Current model: Deeper={}, Wider={}, LR={}, Dropout={}".format(
                    deeper,wider,learning_rate,dropout))
                print()
                save_best_weights = 'question_pairs_weights_deeper={}_wider={}_lr={}_dropout={}.h5'.format(deeper,wider,learning_rate,dropout)
                callbacks = [ModelCheckpoint(save_best_weights, monitor='val_loss', save_best_only=True),
                            EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto'),
                            ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=3)]
                history= model.fit(x_train,
                                  y_train,
                                  batch_size=128,
                                  epochs=100,
                                  validation_split=0.15,
                                  verbose=True,
                                  shuffle=True,
                                  callbacks=callbacks)



Current model: Deeper=False, Wider=True, LR=0.001, Dropout=0.3



C:\Users\edwar\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1435 samples, validate on 254 samples
Epoch 1/100
1435/1435 [==============================] - 8s 6ms/step - loss: 0.2799 - val_loss: 0.0125
Epoch 2/100
1435/1435 [==============================] - 9s 6ms/step - loss: 0.0513 - val_loss: 0.0267
Epoch 3/100
1435/1435 [==============================] - 9s 7ms/step - loss: 0.0347 - val_loss: 0.0264
Epoch 4/100
1435/1435 [==============================] - 9s 7ms/step - loss: 0.0292 - val_loss: 0.0201

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 5/100
1435/1435 [==============================] - 10s 7ms/step - loss: 0.0257 - val_loss: 0.0212
Epoch 6/100
1435/1435 [==============================] - 12s 8ms/step - loss: 0.0258 - val_loss: 0.0197
Epoch 00006: early stopping

Current model: Deeper=False, Wider=True, LR=0.001, Dropout=0.5



C:\Users\edwar\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1435 samples, validate on 254 samples
Epoch 1/100
1435/1435 [==============================] - 14s 10ms/step - loss: 0.4195 - val_loss: 0.0338
Epoch 2/100
1435/1435 [==============================] - 16s 11ms/step - loss: 0.1126 - val_loss: 0.0868
Epoch 3/100
1435/1435 [==============================] - 17s 12ms/step - loss: 0.0803 - val_loss: 0.0404
Epoch 4/100
1435/1435 [==============================] - 17s 12ms/step - loss: 0.0619 - val_loss: 0.0503

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 5/100
1435/1435 [==============================] - 17s 12ms/step - loss: 0.0521 - val_loss: 0.0433
Epoch 6/100
1435/1435 [==============================] - 17s 12ms/step - loss: 0.0471 - val_loss: 0.0353
Epoch 00006: early stopping

Current model: Deeper=False, Wider=False, LR=0.001, Dropout=0.3



C:\Users\edwar\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1435 samples, validate on 254 samples
Epoch 1/100
1435/1435 [==============================] - 17s 12ms/step - loss: 0.3036 - val_loss: 0.0156
Epoch 2/100
1435/1435 [==============================] - 18s 12ms/step - loss: 0.0615 - val_loss: 0.0350
Epoch 3/100
1435/1435 [==============================] - 18s 12ms/step - loss: 0.0372 - val_loss: 0.0280
Epoch 4/100
1435/1435 [==============================] - 18s 13ms/step - loss: 0.0281 - val_loss: 0.0190

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 5/100
1435/1435 [==============================] - 18s 13ms/step - loss: 0.0269 - val_loss: 0.0192
Epoch 6/100
1435/1435 [==============================] - 18s 13ms/step - loss: 0.0223 - val_loss: 0.0214
Epoch 00006: early stopping

Current model: Deeper=False, Wider=False, LR=0.001, Dropout=0.5



C:\Users\edwar\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1435 samples, validate on 254 samples
Epoch 1/100
1435/1435 [==============================] - 18s 13ms/step - loss: 0.4513 - val_loss: 0.0240
Epoch 2/100
1435/1435 [==============================] - 18s 12ms/step - loss: 0.1228 - val_loss: 0.0782
Epoch 3/100
1435/1435 [==============================] - 19s 13ms/step - loss: 0.0784 - val_loss: 0.0494
Epoch 4/100
1435/1435 [==============================] - 18s 13ms/step - loss: 0.0595 - val_loss: 0.0423

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 5/100
1435/1435 [==============================] - 19s 13ms/step - loss: 0.0526 - val_loss: 0.0407
Epoch 6/100
1435/1435 [==============================] - 19s 13ms/step - loss: 0.0492 - val_loss: 0.0402
Epoch 00006: early stopping


In [68]:
# Make predictions with the best weights
deeper=False
wider=True
dropout=0.3
learning_Rate=0.001

model=build_model()

model.load_weights('./question_pairs_weights_deeper={}_wider={}_lr={}_dropout={}.h5'.format(
                    deeper,wider,learning_rate,dropout))
predictions = model.predict(x_test, verbose = True)

299/299 [==============================] - 2s 8ms/step


In [73]:
mse(y_test, predictions)

0.012834476971068648

In [74]:
def unnormalize(price):
    price = price*(max_price-min_price)+min_price
    return(price)

In [75]:
unnorm_predictions=[]
for pred in predictions:
    unnorm_predictions.append(unnormalize(pred))
    
unnorm_y_test = []
for y in y_test:
    unnorm_y_test.append(unnormalize(y))

In [76]:
# Calculate the median absolute error for the predictions
mae(unnorm_y_test, unnorm_predictions)

119.4104005781245

In [77]:
print("Summary of actual opening price changes")
print(pd.DataFrame(unnorm_y_test, columns=[""]).describe())
print()
print("Summary of predicted opening price changes")
print(pd.DataFrame(unnorm_predictions, columns=[""]).describe())

Summary of actual opening price changes
                 
count  299.000000
mean    -7.094101
std    139.532324
min   -541.050782
25%    -87.465332
50%    -10.759766
75%     54.689941
max    673.139648

Summary of predicted opening price changes
                 
count  299.000000
mean    31.592124
std    127.490018
min   -284.840576
25%    -52.418884
50%     18.375427
75%    113.680908
max    576.110474


In [78]:
# Create lists to measure if opening price increased or decreased
direction_pred = []
for pred in unnorm_predictions:
    if pred >= 0:
        direction_pred.append(1)
    else:
        direction_pred.append(0)
direction_test = []
for value in unnorm_y_test:
    if value >= 0:
        direction_test.append(1)
    else:
        direction_test.append(0)

In [79]:
# Calculate if the predicted direction matched the actual direction
direction = acc(direction_test, direction_pred)
direction = round(direction,4)*100
print("Predicted values matched the actual direction {}% of the time.".format(direction))

Predicted values matched the actual direction 49.5% of the time.
